# GDML Explorer

Author: Sam Eriksen

## Purpose
To work out how to explore GDML files to convert to trees for GPU simulations.

Ultimate aim:
1. To develope MutliUnion handling

Additional details at opticks



##### Unusual Libraries Needed
* lxml

### GDML Playing

Start by understanding the structure and using the GDML structure.
Eventually I'll turn this into a class (similar to opticks approach)

A good place to start is the [lxml tutorial](https://lxml.de/tutorial.html)

In [224]:
from lxml import etree
import numpy as np

In [225]:
#
gdml_file = "/global/homes/s/seriksen/Opticks/csg_testing/lzgeom_just_ICV_rmTruss_rmBottomPTFE.gdml"
gdml_e = etree.parse(gdml_file).getroot()
print('gdml element:',gdml_e)

# Get info of element
print('\nGDML structure\n--------------')
for child in gdml_e:
    print(child.tag)

gdml element: <Element gdml at 0x2aaac8c6bd88>

GDML structure
--------------
define
materials
solids
structure
userinfo
setup


## Learning Navigation 
Navigate and view some part of the gdml

In [247]:
# What materials are there
materials = gdml_e.findall('materials/material')
print('number of materials:', np.size(materials), '\n')
for mat in materials:
    attributes = mat.attrib
    print(attributes['name'])

number of materials: 28 

quartz0xe969e0
pcv0xe96140
ss3160xe9bba0
elastomer0xe98e50
steel0xe9b020
lzHVConduitInnerCone0xeb14a0
polyFoam0xe94670
aluminum0xe7e8f0
nicomic0xe99780
inconel0xe9a370
peek0xe931f0
gasXe0xe86fd0
teflon0xe9d950
titanium0xe9da70
sapphire0xe98230
vacuum0xea27f0
Kovar0xeb74a0
alumina0xe98550
liquidXe0xe91d30
dopedLABGd0xe8db20
lzSpecRef0xea16b0
acrylic0xe760f0
lzHVConduitUmbilical0xeb2de0
lzPMTCablingConduit0xeab3b0
lzThermosyphonConduit0xeac950
lzPMTBottomConduit0xeae210
water0xea2ed0
tyvek6000xea1f50


In [251]:
# Solids
solids = gdml_e.findall("solids/*")
print('number of solids:', np.size(solids), '\n')

# Okay that's a lot of solids.. maybe just look at the first few
for solid in solids[0:15]: 
    attributes = solid.attrib
    print(attributes['name'])

number of solids: 12668 

window_solid0x53d7440
photocathode_solid0x53d7840
PMTR5912_solid0x53d7220
window_solid0x53d8110
photocathode_solid0x53d85b0
PMTR5912_solid0x53d7ef0
window_solid0x53d8f60
photocathode_solid0x53d93e0
PMTR5912_solid0x53d8d00
window_solid0x53d9dc0
photocathode_solid0x53da240
PMTR5912_solid0x53d9b60
window_solid0x53dabd0
photocathode_solid0x53db050
PMTR5912_solid0x53da970


In [255]:
# Structure
structure = gdml_e.findall("structure/*")
print('number of structure:', np.size(structure), '\n')

# Okay that's a lot of structure definitions.. maybe just look at the first few
for struct in structure[0:15]: 
    attributes = struct.attrib
    print(attributes['name'])

number of structure: 10407 

PMTR5912Window_log0x53d75b0
PMT7081Photocathode_log0x53d79d0
PMTR5912_log0x53d7390
PMTR5912Window_log0x53d8280
PMT7081Photocathode_log0x53d8740
PMTR5912_log0x53d8060
PMTR5912Window_log0x53d90d0
PMT7081Photocathode_log0x53d9570
PMTR5912_log0x53d8e70
PMTR5912Window_log0x53d9f30
PMT7081Photocathode_log0x53da3b0
PMTR5912_log0x53d9cd0
PMTR5912Window_log0x53dad40
PMT7081Photocathode_log0x53db1e0
PMTR5912_log0x53daae0


# Determine Primitives

For one solid, get the information about it

In [305]:
solid = solids[1]
print('solid: ', solid.attrib['name'], '\n')

l_unit = solid.attrib['lunit']; print('l_unit:', l_unit)
a_unit = solid.attrib['aunit']; print('a_unit:', a_unit)
start_phi = solid.attrib['startphi']; print('start_phi', start_phi)
delta_phi = solid.attrib['deltaphi']; print('delta_phi', delta_phi)
start_theta = solid.attrib['starttheta']; print('start_theta:', start_theta)
delta_theta = solid.attrib['deltatheta']; print('delta_theta:', delta_theta)
r_min = solid.attrib['rmin']; print('r_min:', r_min)
r_max = solid.attrib['rmax']; print('r_max:', r_max)

# Options for other solids
#x = solid.attrib['x']; print('x:', x)
#y = solid.attrib['y']; print('y:', y)
#z = solid.attrib['z']; print('z:', z)

solid:  photocathode_solid0x53d7840 

l_unit: mm
a_unit: deg
start_phi 0
delta_phi 360
start_theta: 0
delta_theta: 47
r_min: 129
r_max: 130


TODO:
1. create default values for the primitives -> lambda
2. Add functions to handle spesific solids

# CSG

Convert XML to CSG


Starting point:

    Using background info from Rossignac, "Blist: A Boolean list formulation of CSG trees"

        * http://www.cc.gatech.edu/~jarek/papers/Blist.pdf  
        * https://www.ics.uci.edu/~pattis/ICS-31/lectures/simplify/lecture.html

        The positive form of a CSG expression is obtained by 
        distributing any negations down the tree to end up  
        with complements in the leaves only.

        * DIFFERNCE -> "-" 
        * UNION -> "+"  "OR"
        * INTERSECTION -> "*"  "AND"
        * COMPLEMENT -> "!" 

        deMorgan Tranformations

            A - B -> A * !B
            !(A*B) -> !A + !B
            !(A+B) -> !A * !B
            !(A - B) -> !(A*!B) -> !A + B

        Example

        (A+B)(C-(D-E))                      (A+B)(C(!D+E))

                       *                                   *
                                                                        
                  +          -                        +          *
                 / \        / \                      / \        / \
                A   B      C    -                   A   B      C    +  
                               / \                                 / \
                              D   E                              !D   E

        test_positivize

            original

                         in                    
                 un              di            
             sp      sp      bo          di    
                                     bo      bo
            positivize

                         in                    
                 un              in            
             sp      sp      bo          un    
                                    !bo      bo